In [1]:
# import libraries
!pip3 install pyserial

import RPi.GPIO as GPIO
import time
import string
import serial


  Using cached https://files.pythonhosted.org/packages/07/bc/587a445451b253b285629263eb51c2d8e9bcea4fc97826266d186f96f558/pyserial-3.5-py2.py3-none-any.whl


In [2]:
#Set GPIO pins to BCM encoding mode
GPIO.setmode(GPIO.BCM)

#Ignore warning messages
GPIO.setwarnings(False)

In [3]:
#Initialize the serial port. If permission issues arise, please open the system terminal and run
#commands to obtain permissions.

# sudo chmod 777 /dev/ttyTHS1
ser = serial.Serial("/dev/ttyTHS1", 115200, timeout = .01)
#print("serial.isOpen()")

In [4]:
#Control servo, where index is the ID number of the servo,
#value is the position of the servo, and s_time is the operating time of the servo.

def Servo_control(index, value, s_time):
    pack1 = 0xff
    pack2 = 0xff
    id = index & 0xff
    len = 0x07
    cmd = 0x03
    addr = 0x2A
    pos1 = (value >> 8) & 0x00ff
    pos2 =  value & 0x00ff 
    time1 = (s_time >> 8) & 0x00ff
    time2 = s_time & 0x00ff
    checknum = (~(id + len + cmd + addr + pos1 + pos2 + time1 + time2)) & 0xff

    data = [pack1, pack2, id, len, cmd, addr, pos1, pos2, time1, time2, checknum]
#     print(bytes(data))
    ser.write(bytes(data))


In [5]:
# set servo ID number
def Servo_Set_ID(index):
    if index < 1 or index > 250:
        return None
    
    pack1 = 0xff
    pack2 = 0xff
    id = 0xfe
    len = 0x04
    cmd = 0x03
    addr = 0x05
    set_id = index & 0xff

    checknum = (~(id + len + cmd + addr + set_id)) & 0xff

    data = [pack1, pack2, id, len, cmd, addr, set_id, checknum]
#     print(bytes(data))
    ser.write(bytes(data))

In [6]:
def main():
    index = 0x01
    
    Servo_Set_ID(index)
    time.sleep(.1)
    
    
    while True:
        Servo_control(index, 3100, 1000)
        time.sleep(2)
        Servo_control(index, 900, 1000)
        time.sleep(2)

In [7]:
try:
    main()
except KeyboardInterrupt:
    pass

ser.close()
GPIO.cleanup()